# Langfuse 프롬프트 관리 

## 환경 설정 및 준비

### (1) Env 환경변수

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

### (2) 기본 라이브러리

In [2]:
import os
from glob import glob
from pprint import pprint
import json
import warnings
warnings.filterwarnings("ignore")

### (3) Langfuse 콜백 핸들러 설정

In [3]:
from langfuse.langchain import CallbackHandler 

# LangChain 콜백 핸들러 생성
langfuse_handler = CallbackHandler()

### (4) Langfuse 클라이언트 설정

In [4]:
from langfuse import get_client

# Langfuse 클라이언트 초기화
langfuse = get_client()

# 연결 테스트
assert langfuse.auth_check()

---

## 프롬프트 관리 개요

Langfuse는 **프롬프트 CMS(Content Management System)** 기능을 제공

- **버전 관리**: 프롬프트의 모든 변경사항을 추적하고 롤백 가능
- **협업**: 팀원들과 함께 프롬프트를 편집하고 관리
- **배포 관리**: 라벨을 통해 코드 변경 없이 환경별 배포
- **성능 모니터링**: 프롬프트 버전별 성능 메트릭 비교
- **실시간 테스트**: 플레이그라운드에서 즉시 테스트 가능

---

## 1. 프롬프트 생성

### 1.1 텍스트 프롬프트 생성

In [5]:
# 텍스트 프롬프트 생성
langfuse.create_prompt(
    name="movie-critic",  # 프롬프트 이름
    type="text",          
    prompt="{{criticLevel}} 영화 평론가로서, {{movie}}를 어떻게 생각하시나요?",
    labels=["production"],       # 프로덕션 레이블
    tags=["movie", "qa", "text"],    # 태그
    config={
        "model": "gpt-4.1-mini",
        "temperature": 0.7,
        "max_tokens": 500
    }
)

### 1.2 챗 프롬프트 생성

In [6]:
# 챗 프롬프트 생성
langfuse.create_prompt(
    name="movie-critic-chat",  # 프롬프트 이름
    type="chat",          
    prompt=[
        {
            "role": "system",
            "content": "당신은 {{criticLevel}} 영화 평론가입니다."
        },
        {
            "role": "user",
            "content": "영화 {{movie}}를 어떻게 생각하시나요?"
        }
    ],
    labels=["production"],       # 프로덕션 레이블
    tags=["movie", "qa", "chat"],    # 태그
    config={
        "model": "gpt-4.1-mini",
        "temperature": 0.7,
        "max_tokens": 500
    }
)

### 1.3 메시지 플레이스홀더가 있는 챗 프롬프트

In [7]:
# 메시지 플레이스홀더를 포함한 챗 프롬프트
langfuse.create_prompt(
    name="movie-critic-with-history",
    type="chat",
    prompt=[
        {
            "role": "system",
            "content": "당신은 {{criticLevel}} 영화 평론가입니다."
        },
        {
            "type": "placeholder",
            "name": "chat_history"  # 대화 히스토리 삽입 지점
        },
        {
            "role": "user",
            "content": "영화 {{movie}}에 대해 어떻게 생각하시나요?"
        }
    ],
    labels=["production"],
    tags=["movie", "qa", "chat", "history"]
)

### **[실습 1]**
텍스트 기반 프롬프트와 chat 기반 프롬프트를 각각 구현하고, Langfuse UI에서 확인하세요.

In [8]:
# 텍스트 프롬프트 생성
# 여기에 코드를 작성하세요

In [9]:
# 챗 프롬프트 생성
# 여기에 코드를 작성하세요

---

## 2. 프롬프트 활용

### 2.1 기본 프롬프트 가져오기

In [10]:
# 프로덕션 버전 가져오기
prompt = langfuse.get_prompt("movie-critic")

# 프롬프트 정보 출력
print(f"모델: {prompt.config['model']}")
print(f"온도: {prompt.config['temperature']}")
print(f"라벨: {prompt.labels}")
print(f"태그: {prompt.tags}")
print(f"프롬프트: {prompt.prompt}")
print("-" * 100)

# 랭체인 호환 프롬프트 출력 (중괄호 1개 사용)
print(prompt.get_langchain_prompt())

모델: gpt-4.1-mini
온도: 0.7
라벨: ['production', 'latest']
태그: ['movie', 'qa', 'text']
프롬프트: {{criticLevel}} 영화 평론가로서, {{movie}}를 어떻게 생각하시나요?
----------------------------------------------------------------------------------------------------
{criticLevel} 영화 평론가로서, {movie}를 어떻게 생각하시나요?


### 2.2 compile 메서드 사용

- compile 메서드로 변수 삽입

In [11]:
# compile 메서드로 변수 삽입
compiled_prompt = prompt.compile(criticLevel="전문가", movie="인셉션")
print(compiled_prompt)

전문가 영화 평론가로서, 인셉션를 어떻게 생각하시나요?


### 2.3 챗 프롬프트 가져오기 및 컴파일

In [13]:
# 챗 프롬프트 가져오기
chat_prompt = langfuse.get_prompt("movie-critic-chat", type="chat")

# 챗 프롬프트 정보 출력
print(f"모델: {chat_prompt.config['model']}")
print(f"온도: {chat_prompt.config['temperature']}")
print(f"라벨: {chat_prompt.labels}")
print(f"태그: {chat_prompt.tags}")
print(f"프롬프트: {chat_prompt.prompt}")
print("-" * 100)

# 랭체인 호환 프롬프트 출력
print(chat_prompt.get_langchain_prompt())

모델: gpt-4.1-mini
온도: 0.7
라벨: ['production', 'latest']
태그: ['movie', 'qa', 'chat']
프롬프트: [{'type': 'message', 'role': 'system', 'content': '당신은 {{criticLevel}} 영화 평론가입니다.'}, {'type': 'message', 'role': 'user', 'content': '영화 {{movie}}를 어떻게 생각하시나요?'}]
----------------------------------------------------------------------------------------------------
[('system', '당신은 {criticLevel} 영화 평론가입니다.'), ('user', '영화 {movie}를 어떻게 생각하시나요?')]


In [14]:
# 챗 프롬프트 컴파일
compiled_chat_prompt = chat_prompt.compile(criticLevel="전문가", movie="인셉션")
print(compiled_chat_prompt)

[{'role': 'system', 'content': '당신은 전문가 영화 평론가입니다.'}, {'role': 'user', 'content': '영화 인셉션를 어떻게 생각하시나요?'}]


### 2.4 메시지 플레이스홀더 활용

In [15]:
# 플레이스홀더가 있는 챗 프롬프트 가져오기
prompt_with_history = langfuse.get_prompt("movie-critic-with-history", type="chat")

# 대화 히스토리 정의
chat_history = [
    {"role": "user", "content": "안녕하세요!"},
    {"role": "assistant", "content": "안녕하세요! 영화에 대해 이야기해볼까요?"}
]

# 변수와 플레이스홀더를 모두 컴파일
compiled_with_history = prompt_with_history.compile(
            criticLevel="전문가",
            movie="인셉션", 
            chat_history=chat_history
        )

for message in compiled_with_history:
    print(message)
    print("-" * 20)

{'role': 'system', 'content': '당신은 전문가 영화 평론가입니다.'}
--------------------
{'role': 'user', 'content': '안녕하세요!'}
--------------------
{'role': 'assistant', 'content': '안녕하세요! 영화에 대해 이야기해볼까요?'}
--------------------
{'role': 'user', 'content': '영화 인셉션에 대해 어떻게 생각하시나요?'}
--------------------


### **[실습 2]**
"movie-critic-chat" 프롬프트를 Langfuse에서 가져와서 내용을 출력하고, compile 메서드를 사용해 변수에 적절한 값을 추가해보세요.

In [ ]:
# chat 프롬프트 가져오기 및 컴파일
# 여기에 코드를 작성하세요

In [ ]:
# chat 프롬프트 가져오기 및 컴파일
chat_prompt = langfuse.get_prompt("movie-critic-chat", type="chat")

# 대화 히스토리 정의
chat_history = [
    {"role": "user", "content": "안녕하세요!"},
    {"role": "assistant", "content": "안녕하세요! 영화에 대해 이야기해볼까요?"}
]

# 변수와 플레이스홀더를 모두 컴파일
compiled_chat_prompt = chat_prompt.compile(
    criticLevel="전문가", 
    movie="인셉션"
)

for message in compiled_chat_prompt:
    print(message)
    print("-" * 20)

---

## 3. 프롬프트 버전 관리

### 3.1 새로운 버전 생성

In [16]:
# 새로운 버전 생성 (같은 이름 사용)
langfuse.create_prompt(
    name="movie-critic",  # 같은 이름 사용
    type="text",          
    prompt="당신은 {{criticLevel}} 영화 평론가입니다.\n\n영화 {{movie}}에 대한 상세한 분석을 제공해주세요. 연출, 연기, 스토리, 시각적 효과를 포함하여 평가해주세요.",
    labels=["production"],       # 프로덕션 레이블
    tags=["movie", "qa", "text", "detailed"],    # 태그 업데이트
    config={
        "model": "gpt-4.1",  # 모델 업그레이드
        "temperature": 0.7,
        "max_tokens": 1000  # 토큰 수 증가
    }
)

### 3.2 특정 버전 가져오기

In [17]:
# 특정 버전 가져오기
prompt_v1 = langfuse.get_prompt("movie-critic", version=1)
prompt_v2 = langfuse.get_prompt("movie-critic", version=2)

# 버전별 비교
print(f"V1 프롬프트: {prompt_v1.prompt}")
print(f"V2 프롬프트: {prompt_v2.prompt}")
print(f"V1 모델: {prompt_v1.config['model']}")
print(f"V2 모델: {prompt_v2.config['model']}")

V1 프롬프트: {{criticLevel}} 영화 평론가로서, {{movie}}를 어떻게 생각하시나요?
V2 프롬프트: 당신은 {{criticLevel}} 영화 평론가입니다.

영화 {{movie}}에 대한 상세한 분석을 제공해주세요. 연출, 연기, 스토리, 시각적 효과를 포함하여 평가해주세요.
V1 모델: gpt-4.1-mini
V2 모델: gpt-4.1


### 3.3 라벨 관리

In [18]:
# 특정 라벨로 프롬프트 생성 (같은 이름을 사용하면 새로운 버전으로 생성됨)
langfuse.create_prompt(
    name="movie-critic-chat",
    type="chat",
    prompt=[
        {
            "role": "system",
            "content": "당신은 {{criticLevel}} 영화 평론가입니다. 상세하고 전문적인 분석을 제공해주세요."
        },
        {
            "role": "user",
            "content": "영화 {{movie}}에 대한 평론을 작성해주세요."
        }
    ],
    labels=["staging"],  # staging 환경용
    tags=["movie", "qa", "chat", "detailed"]
)

# 라벨별 프롬프트 가져오기
prompt_production = langfuse.get_prompt("movie-critic-chat", label="production")
prompt_staging = langfuse.get_prompt("movie-critic-chat", label="staging")
prompt_latest = langfuse.get_prompt("movie-critic-chat", label="latest")

In [19]:
# 라벨별 프롬프트 출력
print(f"Production 프롬프트: {prompt_production.prompt}")
print("-" * 100)
print(f"Staging 프롬프트: {prompt_staging.prompt}")
print("-" * 100)
print(f"Latest 프롬프트: {prompt_latest.prompt}")

Production 프롬프트: [{'type': 'message', 'role': 'system', 'content': '당신은 {{criticLevel}} 영화 평론가입니다.'}, {'type': 'message', 'role': 'user', 'content': '영화 {{movie}}를 어떻게 생각하시나요?'}]
----------------------------------------------------------------------------------------------------
Staging 프롬프트: [{'type': 'message', 'role': 'system', 'content': '당신은 {{criticLevel}} 영화 평론가입니다. 상세하고 전문적인 분석을 제공해주세요.'}, {'type': 'message', 'role': 'user', 'content': '영화 {{movie}}에 대한 평론을 작성해주세요.'}]
----------------------------------------------------------------------------------------------------
Latest 프롬프트: [{'type': 'message', 'role': 'system', 'content': '당신은 {{criticLevel}} 영화 평론가입니다. 상세하고 전문적인 분석을 제공해주세요.'}, {'type': 'message', 'role': 'user', 'content': '영화 {{movie}}에 대한 평론을 작성해주세요.'}]


### 3.4 라벨 업데이트

In [20]:
# 기존 프롬프트 버전의 라벨 업데이트
langfuse.update_prompt(
    name="movie-critic-chat",
    version=2,
    new_labels=["production", "v2-stable"]
)

Prompt_Chat(prompt=[ChatMessageWithPlaceholders_Chatmessage(role='system', content='당신은 {{criticLevel}} 영화 평론가입니다. 상세하고 전문적인 분석을 제공해주세요.', type='chatmessage'), ChatMessageWithPlaceholders_Chatmessage(role='user', content='영화 {{movie}}에 대한 평론을 작성해주세요.', type='chatmessage')], name='movie-critic-chat', version=2, config={}, labels=['production', 'v2-stable', 'staging', 'latest'], tags=['movie', 'qa', 'chat', 'detailed'], commit_message=None, resolution_graph=None, type='chat', updatedAt='2025-09-09T00:37:26.893Z', id='7e8dc621-6a47-4e9b-884a-cdf20c3584ef', isActive=None, createdBy='API', projectId='cmfbt9zku0b94ad077nt3sxeh', createdAt='2025-09-09T00:35:51.919Z')

### **[실습 3]**
"movie-critic-chat" 프롬프트를 수정하고, labels 속성은 "staging"으로 지정한 후, staging 버전을 가져와서 내용을 출력하세요.

In [21]:
# staging 라벨 생성
# 여기에 코드를 작성하세요

langfuse.create_prompt(
    name="movie-critic-chat",
    type="chat",
    prompt=[
        {
            "role": "system",
            "content": "당신은 {{criticLevel}} 영화 평론가입니다. 항상 친근하게 말을 건네며 시작합니ㅏ. 상세하고 전문적인 분석을 제공해주세요."
        },
        {
            "role": "user",
            "content": "영화 {{movie}}에 대한 평론을 작성해주세요."
        }
    ],
    labels=["staging"],  # staging 환경용
    tags=["movie", "qa", "chat", "detailed"]
)

# staging 라벨 가져오기
# 여기에 코드를 작성하세요
prompt_staging = langfuse.get_prompt("movie-critic-chat", label="staging")
print(f"Staging 프롬프트: {prompt_staging.prompt}")
print("-" * 100)

Staging 프롬프트: [{'type': 'message', 'role': 'system', 'content': '당신은 {{criticLevel}} 영화 평론가입니다. 항상 친근하게 말을 건네며 시작합니ㅏ. 상세하고 전문적인 분석을 제공해주세요.'}, {'type': 'message', 'role': 'user', 'content': '영화 {{movie}}에 대한 평론을 작성해주세요.'}]
----------------------------------------------------------------------------------------------------


In [22]:
# staging 라벨 생성
# 여기에 코드를 작성하세요
langfuse.create_prompt(
    name="movie-critic-chat",
    type="chat",
    prompt=[
        {
            "role": "system",
            "content": "당신은 {{criticLevel}} 영화 평론가입니다. 일반적이고 상세하며 전문적인 분석을 제공해주세요."
        },
        {
            "role": "user",
            "content": "영화 {{movie}}에 대한 평론을 작성해주세요."
        }
    ],
    labels=["staging"],  # staging 환경용
    tags=["movie", "qa", "chat", "detailed"]
)

# staging 라벨 가져오기
prompt_staging = langfuse.get_prompt(
    name="movie-critic-chat",
    label="staging"
)

# 라벨별 프롬프트 출력
print(f"Staging 프롬프트: {prompt_staging.prompt}")

Staging 프롬프트: [{'type': 'message', 'role': 'system', 'content': '당신은 {{criticLevel}} 영화 평론가입니다. 일반적이고 상세하며 전문적인 분석을 제공해주세요.'}, {'type': 'message', 'role': 'user', 'content': '영화 {{movie}}에 대한 평론을 작성해주세요.'}]


---

## 4. LangChain과의 통합

### 4.1 텍스트 프롬프트와 LangChain 통합

In [23]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# Langfuse 프롬프트를 LangChain과 통합
prompt = langfuse.get_prompt("movie-critic", label="production")

langchain_prompt = PromptTemplate.from_template(
    prompt.get_langchain_prompt(),
    metadata={"langfuse_prompt": prompt},  # Langfuse 자동 링크를 위한 메타데이터
)

# 모델 초기화 (프롬프트 설정 사용)
model = ChatOpenAI(
    model=prompt.config.get("model", "gpt-4.1-mini"),
    temperature=prompt.config.get("temperature", 0.7),
    max_completion_tokens=prompt.config.get("max_tokens", 500)
)

# 체인 생성 및 실행
chain = langchain_prompt | model
response = chain.invoke(
    input={"criticLevel": "전문가", "movie": "인셉션"},
    config={"callbacks": [langfuse_handler]}  # Langfuse 트레이싱을 위한 콜백
)

print(response.content)

네, 전문가 영화 평론가의 시각에서 크리스토퍼 놀란 감독의 **『인셉션』(Inception, 2010)**을 상세하게 분석하겠습니다.

---

## 1. 연출

크리스토퍼 놀란은 본작에서 치밀하면서도 독창적인 연출력을 선보입니다. 꿈과 현실, 다중 레이어의 세계를 유기적으로 엮어내며, 관객이 자연스럽게 복잡한 구조에 몰입하도록 유도합니다. 특히 꿈의 시간 흐름과 현실의 시간 흐름의 차이를 교차 편집으로 표현한 점은 탁월합니다. 액션과 드라마, 서스펜스가 완벽하게 조화를 이루며, 한 순간도 긴장감을 놓치지 않습니다. ‘꿈 속의 꿈’이라는 난해한 설정을 시각적으로 명확하게 구분해내는 솜씨도 인상적입니다.

---

## 2. 연기

레오나르도 디카프리오(돔 코브 역)는 트라우마에 시달리는 복합적인 인물을 섬세하게 연기하며, 관객이 그의 감정에 공감하도록 만듭니다. 마리옹 꼬띠아르(맬 역)는 미스터리하면서도 강렬한 존재감으로 극의 긴장감을 극대화합니다. 조셉 고든-레빗, 엘렌 페이지, 톰 하디 등 조연 배우들도 각자의 개성을 뚜렷이 드러내며 팀의 역동성을 완성합니다. 전체적으로 연기 앙상블이 매우 탄탄해, 현실과 환상이 뒤섞인 세계에서도 감정의 리얼리티를 놓치지 않습니다.

---

## 3. 스토리

『인셉션』의 스토리는 ‘꿈을 통해 생각(아이디어)을 주입한다’는 독특한 설정에서 출발합니다. 영화는 한 사람의 내면 세계와 무의식, 죄책감, 상실 등의 심리적 요소를 SF와 범죄 스릴러 장르에 절묘하게 녹여냅니다. 복잡한 플롯 구조에도 불구하고, 인물의 감정선과 주제 의식이 분명하게 드러나 관객의 이해와 몰입을 돕습니다. 엔딩의 ‘토템’ 장면은 현실과 환상, 진실과 자기기만에 대한 질문을 남기며 오랜 여운을 줍니다.

---

## 4. 시각적 효과

시각적 효과는 『인셉션』의 가장 큰 미덕 중 하나입니다. 파리의 거리가 접히고, 중력이 뒤틀리며, 건물이 폭발하는 등 상상력을 극한으로 확장한 비주얼은 새로운 차원의 ‘꿈’을 구현합니다. 컴퓨터 그래픽과 실사 촬영이 

### 4.2 챗 프롬프트와 LangChain 통합

In [24]:
from langchain_core.prompts import ChatPromptTemplate

# 챗 프롬프트 통합
chat_prompt = langfuse.get_prompt("movie-critic-chat", label="production", type="chat")

langchain_chat_prompt = ChatPromptTemplate.from_messages(
    chat_prompt.get_langchain_prompt()
)
langchain_chat_prompt.metadata = {"langfuse_prompt": chat_prompt}

# 체인 실행
chain = langchain_chat_prompt | model
response = chain.invoke(
    input={"criticLevel": "전문가", "movie": "인셉션"},
    config={"callbacks": [langfuse_handler]}
)

print(response.content)

크리스토퍼 놀란 감독의 2010년작 **<인셉션(Inception)>**은 현대 SF 스릴러의 새로운 지평을 연 작품으로, 영화사에서 드물게 ‘꿈’이라는 추상적 개념을 정교한 플롯과 시각적 스펙터클로 구현했다는 점에서 높이 평가받는다. 이 영화는 단순한 오락영화를 넘어, 무의식과 현실, 죄책감과 구원, 인간 심리의 복잡성을 다루며 관객에게 깊은 사유의 경험을 제공한다.

### 1. 서사 구조와 내러티브의 혁신성

**<인셉션>**의 서사는 ‘꿈속의 꿈’이라는 중첩된 구조로 이루어져 있다. 주인공 돔 코브(레오나르도 디카프리오 분)가 각기 다른 인물들과 팀을 이뤄 타인의 무의식에 침투, 특정 아이디어를 심는 ‘인셉션’ 임무를 수행하는 과정은 다층적 내러티브의 진수를 보여준다. 각 층위(현실-첫 번째 꿈-두 번째 꿈-림보)는 시공간의 법칙이 다르고, 시간의 속도 역시 계층별로 달라진다. 놀란 감독은 이러한 복잡성을 혼란스럽지 않게 전달하기 위해, 색채, 음악, 편집의 리듬, 액션의 톤을 세밀하게 조율한다. 이는 관객이 각 층위의 경계와 규칙을 자연스럽게 인지하도록 돕는다.

### 2. 주제와 심리적 깊이

영화는 단순히 ‘꿈을 꾸는 사람들’의 이야기에서 멈추지 않는다. 놀란은 코브의 내면적 상처와 죄책감을 이야기의 핵심 동력으로 삼는다. 코브와 그의 아내 멀(마리옹 꼬띠아르 분) 사이의 비극적 사랑, 그리고 멀의 죽음을 둘러싼 죄책감은 꿈의 세계가 단순한 미로가 아니라, 인간 심리의 무의식이 투영된 공간임을 상징한다. 꿈의 구조에 대한 치밀한 설정이 오히려 인간 내면의 불안과 상처, 구원을 향한 갈망을 더 극적으로 드러낸다.

### 3. 시각적·청각적 스타일

**<인셉션>**은 놀란 특유의 현실과 비현실의 경계에 선 시각적 스타일이 빛을 발한다. 파리의 거리들이 접히고, 중력이 뒤틀리며, 꿈의 세계가 무너지는 장면은 CG와 실제 촬영의 경계에서 관객에게 ‘꿈의 논리’와 ‘불가능한 현실감’을 동시에 선사한다. 한스 짐머의 음악은 영화의 긴장감과 감정선을 견고

### 4.3 플레이스홀더가 있는 프롬프트와 LangChain 통합

In [25]:
from langchain_core.prompts import MessagesPlaceholder

# 플레이스홀더가 있는 프롬프트 가져오기
prompt_with_history = langfuse.get_prompt("movie-critic-with-history", type="chat")

# LangChain 호환 프롬프트로 변환 (미해결 플레이스홀더는 MessagesPlaceholder로 변환)
langchain_prompt_with_placeholder = ChatPromptTemplate.from_messages(
    prompt_with_history.get_langchain_prompt()
)
langchain_prompt_with_placeholder.metadata = {"langfuse_prompt": prompt_with_history}

# chain 생성
chain = langchain_prompt_with_placeholder | model

# 실행 시 플레이스홀더 값 제공
chat_history = [
    {"role": "user", "content": "안녕하세요! 영화 예산에 대해서 이야기해볼까요?"},
    {"role": "assistant", "content": "안녕하세요! 영화 예산에 대해 어떻게 도와드릴까요?"}
]

response = chain.invoke({
    "criticLevel": "전문가", 
    "movie": "인셉션",
    "chat_history": chat_history
}, config={"callbacks": [langfuse_handler]})  # Langfuse 트레이싱을 위한 콜백

print(response.content)

Placeholders ['chat_history'] have not been resolved. Pass them as keyword arguments to compile().


크리스토퍼 놀란 감독의 2010년 작품인 **《인셉션(Inception)》**은 현대 SF 영화의 걸작으로 자주 언급됩니다. 영화는 ‘꿈속의 꿈’이라는 독창적이고 복잡한 구조를 통해, 인간의 무의식과 현실, 그리고 아이디어의 힘에 대한 깊은 탐구를 보여줍니다.

**영화적 완성도**  
놀란 감독의 특유의 치밀한 플롯 구성과 시간 구조의 실험이 돋보입니다. 관객이 끊임없이 ‘지금이 현실인가, 꿈인가’를 고민하게 만드는 전개와, 마지막 회전하는 팽이로 대표되는 열린 결말까지, 상상력을 자극하는 요소들이 뛰어납니다.

**연출과 연기**  
레오나르도 디카프리오, 마리옹 꼬띠아르, 조셉 고든 레빗 등 배우들의 열연도 인상적입니다. 각 인물의 감정선이 복잡한 이야기를 따라가면서도 자연스럽게 전달되어, 관객의 몰입도를 높입니다.

**시각적, 기술적 측면**  
시각효과(VFX)와 실사 촬영의 조화가 훌륭합니다. 파리 거리가 접히고, 무중력 액션 신 등은 영화사에 남을 명장면입니다. 한스 짐머의 음악 역시 긴장감과 감정의 깊이를 더합니다.

**예산과 흥행**  
약 1억 6천만 달러의 제작비가 투입된 블록버스터로, 전 세계적으로 8억 달러 이상의 흥행 수익을 올렸습니다. 높은 예산을 아트적인 실험과 상업적 재미, 두 마리 토끼를 잡는 데 성공적으로 사용한 사례라 평가할 수 있습니다.

**총평**  
《인셉션》은 상상력, 기술력, 서사, 연기 등 모든 측면에서 높은 완성도를 보인 작품입니다. 오락성과 예술성을 겸비한, 21세기 최고의 SF 영화 중 하나로 꼽을 만합니다.


### **[실습 4]**
앞에서 정의한 텍스트 기반 프롬프트를 가져와서 LangChain과 통합하여 트레이싱을 실행하고, Langfuse UI에서 결과를 확인하세요.


In [ ]:
# 여기에 코드를 추가하세요.

In [ ]:
# 특정 라벨 가져오기
prompt_staging = langfuse.get_prompt("movie-critic", label="latest")  # production, latest

# 프롬프트 출력
print(f"모델: {prompt_staging.config['model']}")
print(f"온도: {prompt_staging.config['temperature']}")
print(f"라벨: {prompt_staging.labels}")
print(f"프롬프트: {prompt_staging.prompt}")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# Langchain과 통합 - 'chat' 프롬프트
langchain_prompt = ChatPromptTemplate.from_template(
    prompt_staging.get_langchain_prompt(type="chat"),
)
langchain_prompt.metadata = {"langfuse_prompt": prompt_staging}

print(langchain_prompt.format(criticLevel="비평가", movie="인셉션"))

In [ ]:
from langchain_openai import ChatOpenAI

# ChatOpenAI 모델 초기화 (프롬프트 설정에서 가져온 값 사용)
model = ChatOpenAI(
    model=prompt.config.get("model", "gpt-4.1-mini"),
    temperature=prompt.config.get("temperature", 0.7)
)

# 체인 생성 및 실행
chain = langchain_prompt | model
response = chain.invoke(
    input={"criticLevel": "전문가", "movie": "인셉션"},
    config={"callbacks": [langfuse_handler]}  # 콜백 핸들러 추가
)

# 응답 출력
print(response.content)

### **[실습 5]**
앞에서 정의한 chat 기반 프롬프트를 가져와서 LangChain과 통합하여 트레이싱을 실행하고, Langfuse UI에서 결과를 확인하세요.

In [ ]:
# 여기에 코드를 추가하세요.